In [1]:
instance_uri = "https://api.stage.sinopia.io/resource/6fb58472-99b8-4711-8300-666578733be5"
# change the instance_uri to the URI of the instance that is linked to the work resource you want to serialize for Alma RDF/XML.
# add api key and region for posting the resource to Alma
# https://github.com/LD4P/ils-middleware/wiki/Alma-APIs-in-Airflow

alma_api_key = "your_api_key"
uri_region = "https://api-na.hosted.exlibrisgroup.com"

In [2]:
!pip install rdflib
!pip install lxml
!pip install requests

In [3]:
import requests
from rdflib import Graph, Namespace, URIRef, RDF
from rdflib.namespace import RDF
from lxml import etree as ET
from name_space.alma_ns import alma_namespaces
from copy import deepcopy


In [4]:
instance_uri = URIRef(instance_uri)
work_uri = None
instance_graph = Graph()
work_graph = Graph()
instance_graph.parse(instance_uri)


<Graph identifier=N397f522c78154635aa9831f9d4ad5626 (<class 'rdflib.graph.Graph'>)>

In [5]:
 # Define the bf namespace
bf = Namespace("http://id.loc.gov/ontologies/bibframe/")

for prefix, url in alma_namespaces:
    work_graph.bind(prefix, url)
work_uri = instance_graph.value(
    subject=URIRef(instance_uri), predicate=bf.instanceOf
        )
work_uri = URIRef(work_uri)
# Explicitly state that work_uri is of type bf:Work
work_graph.add((work_uri, RDF.type, bf.Work))

# parse the work graph
work_graph.parse(work_uri)

# serialize the work graph
bfwork_alma_xml = work_graph.serialize(format="pretty-xml", encoding="utf-8")
tree = ET.fromstring(bfwork_alma_xml)
print(ET.tostring(tree, pretty_print=True).decode("utf-8"))
# save the work graph to a file
with open("work_graph.xml", "wb") as f:
    f.write(bfwork_alma_xml)
    

<rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
  <bf:Work rdf:about="https://api.stage.sinopia.io/resource/5c702b5a-1f2a-4aff-bdfb-c132c14860f7">
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/MusicAudio"/>
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Monograph"/>
    <sinopia:hasResourceTemplate>mapswg:bf2:SoundRecording:Work</sinopia:hasResourceTemplate>
    <bf:title>
      <bf:Title rdf:nodeID="f73ab6e88c65a41d9b6cebdf1583a888cb1">
        <bf:mainTitle xml:lang="en">Together again for the first time</bf:mainTitle>
      </bf:Title>
    </bf:title>
    <bf:contribution>
      <bf:Contribution rdf:nodeID="f73ab6e88c65a41d9b6cebdf1583a888cb2">
        <bf:agent rdf:resource="http://id.loc.gov/authorities/names/n50014544"/>
        <bf

/Users/jimhahn/Documents/GitHub/sinopia2alma_processing/.conda/lib/python3.11/site-packages/rdflib/plugins/serializers/rdfxml.py:280: UserWarning: Assertions on rdflib.term.BNode('f73ab6e88c65a41d9b6cebdf1583a888cb10') other than RDF.first and RDF.rest are ignored ... including RDF.List
  self.predicate(predicate, object, depth + 1)


In [6]:
from lxml import etree as ET
from copy import deepcopy

# Parse the XML file
tree = ET.parse('work_graph.xml')
work_graph = tree.getroot()

# Define namespaces
namespaces = {
    'bf': 'http://id.loc.gov/ontologies/bibframe/',
    'bflc': 'http://id.loc.gov/ontologies/bflc/',
    'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
    'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',   
}

# Find all bf:Work elements
works = work_graph.xpath('//bf:Work', namespaces=namespaces)
print(f"Found {len(works)} bf:Work elements")

# Create a list to keep track of works to remove
works_to_remove = []

for work in works:
    work_about = work.attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about')
    print(f"Processing bf:Work with rdf:about={work_about}")

    if work_about:
        # Find the bf:relatedTo elements that are directly linked to a bf:Work element
        related_to_elements = work_graph.xpath(f'//bf:relatedTo[@rdf:resource="{work_about}"]', namespaces=namespaces)
        print(f"Found {len(related_to_elements)} bf:relatedTo elements for rdf:about={work_about}")

        cloned = False  # Flag to track if the work has been cloned

        for related_to in related_to_elements:
            # Check if the bf:relatedTo element is within a bflc:Relationship
            relationship = related_to.getparent()
            if relationship.tag in ('{http://id.loc.gov/ontologies/bflc/}Relationship', '{http://id.loc.gov/ontologies/bibframe/}Relationship'):
                # Check if the bflc:Relationship or bf:Relationship contains a bf:Work element
                if relationship.find('bf:relatedTo/bf:Work', namespaces=namespaces) is not None:
                    print(f"bf:relatedTo element is within a bflc:Relationship that contains a bf:Work element")

                    # Remove the rdf:resource attribute from the bf:relatedTo element
                    related_to.attrib.pop('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource', None)

                    # Check if the bf:Work element already exists within bf:relatedTo
                    existing_work = related_to.find('bf:Work', namespaces=namespaces)
                    if existing_work is None:
                        # Clone the bf:Work element and append it under the bf:relatedTo element
                        cloned_work = deepcopy(work)
                        related_to.append(cloned_work)
                        cloned = True  # Set the flag to True
                        print(f"Cloned bf:Work element appended under bf:relatedTo")

        # Add the original bf:Work element to the list of works to remove only if it was cloned
        if cloned:
            works_to_remove.append(work)

# Remove the original bf:Work elements
for work in works_to_remove:
    work.getparent().remove(work)
    print(f"Removed original bf:Work element with rdf:about={work.attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about')}")

# Print the modified XML
modified_xml = ET.tostring(work_graph, pretty_print=True).decode()
print(modified_xml)

# Save the modified XML to a file
with open('bfwork_alma.xml', 'wb') as f:
    f.write(ET.tostring(work_graph, pretty_print=True))
    f.close()

Found 1 bf:Work elements
Processing bf:Work with rdf:about=https://api.stage.sinopia.io/resource/5c702b5a-1f2a-4aff-bdfb-c132c14860f7
Found 0 bf:relatedTo elements for rdf:about=https://api.stage.sinopia.io/resource/5c702b5a-1f2a-4aff-bdfb-c132c14860f7
<rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
  <bf:Work rdf:about="https://api.stage.sinopia.io/resource/5c702b5a-1f2a-4aff-bdfb-c132c14860f7">
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/MusicAudio"/>
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Monograph"/>
    <sinopia:hasResourceTemplate>mapswg:bf2:SoundRecording:Work</sinopia:hasResourceTemplate>
    <bf:title>
      <bf:Title rdf:nodeID="f73ab6e88c65a41d9b6cebdf1583a888cb1">
        <bf:mainTitle xml:lang="en">Together again fo

In [7]:
# open the file and parse the XML
tree = ET.parse("bfwork_alma.xml")
xslt = ET.parse("xsl/normalize-work-sinopia2alma.xsl")
transform = ET.XSLT(xslt)
alma_xml = transform(tree)
alma_xml = ET.tostring(
        alma_xml, pretty_print=True, encoding="utf-8"
        )
# save the xml to a file
with open("alma-work.xml", "wb") as f:
    f.write(alma_xml)
print(alma_xml.decode("utf-8"))

<bib>
  <record_format>lcbf_work</record_format>
  <suppress_from_publishing>false</suppress_from_publishing>
  <record>
    <rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
  <bf:Work rdf:about="https://api.stage.sinopia.io/resource/5c702b5a-1f2a-4aff-bdfb-c132c14860f7" xmlns:bf="http://id.loc.gov/ontologies/bibframe/">
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/MusicAudio"/>
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Monograph"/>
    <sinopia:hasResourceTemplate>mapswg:bf2:SoundRecording:Work</sinopia:hasResourceTemplate>
    <bf:title>
      <bf:Title rdf:nodeID="f73ab6e88c65a41d9b6cebdf1583a888cb1">
        <bf:mainTitle xml:lang="en">Together again for the first time</bf:mainTitle>
      </bf:Title>
    </bf:title>
    <bf:cont

In [11]:
# handle 400, an update to the record in Alma
def parse_400(result):
    xml_response = ET.fromstring(result)
    xslt = ET.parse("xsl/put_mms_id.xsl")
    transform = ET.XSLT(xslt)
    result_tree = transform(xml_response)
    put_mms_id_str = str(result_tree)
    print(f"put_mms_id_str: {put_mms_id_str}")
    return put_mms_id_str

# post the work to Alma
def NewWorktoAlma():
    with open("alma-work.xml", "rb") as f:
        data = f.read()

        alma_uri = (
            uri_region
            + "/almaws/v1/bibs?"
            + "from_nz_mms_id=&from_cz_mms_id=&normalization=&validate=false"
            + "&override_warning=true&check_match=false&import_profile=&apikey="
            + alma_api_key
        )
        # post to alma
        alma_result = requests.post(
            alma_uri,
            headers={
                "Content-Type": "application/xml; charset=utf-8",
                "Accept": "application/xml",
                "x-api-key": alma_api_key,
            },
            data=data,
        )
        print(f"alma result: {alma_result.status_code}\n{alma_result.text}")
        result = alma_result.content
        status = alma_result.status_code
        if status == 200:
            xml_response = ET.fromstring(result)
            mms_id = xml_response.xpath("//mms_id/text()")
            print(f"Created record {mms_id}")
        elif status == 400:
            # run xslt on the result in case the response is 400 and we need to update the record
            put_mms_id_str = parse_400(result)
            alma_update_uri = (
                uri_region
                + "/almaws/v1/bibs/"
                + put_mms_id_str
                + "?normalization=&validate=false&override_warning=true"
                + "&override_lock=true&stale_version_check=false&cataloger_level=&check_match=false"
                + "&apikey="
                + alma_api_key
            )
            putWorkToAlma(
                alma_update_uri,
                data,
            )
        else:
            raise Exception(f"Unexpected status code from Alma API: {status}")

# update the instance in Alma
def putWorkToAlma(
    alma_update_uri,
    data,
):
    put_update = requests.put(
        alma_update_uri,
        headers={
            "Content-Type": "application/xml; charset=UTF-8",
            "Accept": "application/xml",
        },
        data=data,
    )
    print(f"put update: {put_update.status_code}\n{put_update.text}")
    put_update_status = put_update.status_code
    result = put_update.content
    xml_response = ET.fromstring(result)
    put_mms_id = xml_response.xpath("//mms_id/text()")
    match put_update_status:
        case 200:
            print(f"Updated record {put_mms_id}")
        case 500:
            raise Exception(f"Internal server error from Alma API: {put_update_status}")
        case _:
            raise Exception(
                f"Unexpected status code from Alma API: {put_update_status}"
            )

# Call the function
NewWorktoAlma()

alma result: 200
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><bib><mms_id>9779515824103681</mms_id><record_format>lc_bf_work</record_format><linked_record_id/><title>Together again for the first time</title><author>Tormé, Mel,</author><date_of_publication>5|||</date_of_publication><holdings link="https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/9779515824103681/holdings"/><created_by>exl_api</created_by><created_date>2024-10-30Z</created_date><last_modified_by>exl_api</last_modified_by><last_modified_date>2024-10-30Z</last_modified_date><suppress_from_publishing>false</suppress_from_publishing><suppress_from_external_search>false</suppress_from_external_search><suppress_from_metadoor>false</suppress_from_metadoor><sync_with_oclc>NONE</sync_with_oclc><sync_with_libraries_australia>NONE</sync_with_libraries_australia><originating_system>01UPENN_INST</originating_system><originating_system_id>5c702b5a-1f2a-4aff-bdfb-c132c14860f7</originating_system_id><brief_level desc="